In [106]:
import sys
import os
import math
import pandas as pd
from collections import Counter

# --- Configuración de Rutas ---
# Se agrega la ruta del dataset 'libreriari' al path del sistema para permitir
# la importación de los módulos personalizados desarrollados para el examen.
RUTA_LIBRERIA = '/kaggle/input/libreriari' 

if RUTA_LIBRERIA not in sys.path:
    sys.path.append(RUTA_LIBRERIA)

In [107]:
# --- 1. Importación de Módulos de LibreriaRI ---
try:
    from LibreriaRI.cleaning import (
        normalize_unicode, to_lower, remove_urls, remove_emails, 
        remove_html_tags, remove_digits, remove_punctuation, 
        strip_accents, normalize_whitespace
    )
    from LibreriaRI.tokenization import (
        simple_tokenize, filter_alpha, filter_min_length, 
        remove_stopwords 
    )
except ImportError as error:
    print(f"Error en la importación de la librería: {error}")

# --- 2. Configuración de Stemming (NLTK) ---
# Se utiliza el algoritmo PorterStemmer para reducir las palabras a su raíz léxica,
# permitiendo agrupar variantes morfológicas (ej: running -> run).
from nltk.stem import PorterStemmer
estematizador = PorterStemmer()

# --- 3. Carga de Stopwords en Inglés ---
def cargar_stopwords_externas(ruta_archivo):
    """
    Lee un archivo de texto plano y genera un conjunto (set) de palabras vacías
    para optimizar la velocidad de búsqueda y filtrado.
    """
    palabras_vacias = set()
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            for linea in archivo:
                palabra = linea.strip()
                if palabra: palabras_vacias.add(palabra)
        print(f"Lista de exclusión cargada: {len(palabras_vacias)} términos.")
    except FileNotFoundError:
        print(f"Error: Archivo no encontrado en {ruta_archivo}")
    return palabras_vacias

# Definición de la ruta del archivo de stopwords subido al dataset
RUTA_STOPWORDS = '/kaggle/input/libreriari/english_stopwords.txt'
mis_stopwords = cargar_stopwords_externas(RUTA_STOPWORDS)

Lista de exclusión cargada: 91 términos.


In [108]:
def procesar_texto(texto_crudo, mostrar_traza=False):
    """
    Aplica secuencialmente limpieza, tokenización, filtrado y stemming.
    
    Parámetros:
        mostrar_traza (bool): Si es True, imprime el estado intermedio del texto
                              para fines de auditoría y validación del proceso.
    """
    if pd.isna(texto_crudo): return []
    
    # 1. Limpieza y Normalización (Uso de LibreriaRI)
    if mostrar_traza: print(f"\n[AUDITORÍA] Texto Original: {str(texto_crudo)[:60]}...")
    
    texto = normalize_unicode(texto_crudo)
    texto = to_lower(texto)
    texto = remove_urls(texto)
    texto = remove_emails(texto)
    texto = remove_html_tags(texto)
    texto = remove_digits(texto)
    texto = remove_punctuation(texto)
    texto = strip_accents(texto)
    texto = normalize_whitespace(texto)
    
    # 2. Tokenización
    tokens = simple_tokenize(texto)
    tokens = filter_alpha(tokens)
    tokens = filter_min_length(tokens, min_len=2)
    
    # 3. Filtrado de Stopwords (Lista en inglés cargada previamente)
    tokens = remove_stopwords(tokens, mis_stopwords)
    if mostrar_traza: print(f"[AUDITORÍA] Pre-Stemming: {tokens[:10]}...")
    
    # 4. Stemming (Reducción a raíz)
    # Se aplica el estematizador a cada token para mejorar el recall del sistema.
    tokens_procesados = [estematizador.stem(token) for token in tokens]
    
    if mostrar_traza: 
        print(f"[AUDITORÍA] Post-Stemming: {tokens_procesados[:10]}... (Total: {len(tokens_procesados)})")
    
    return tokens_procesados

# Verificación del funcionamiento del pipeline
print("Prueba de funcionamiento del pipeline:")
_ = procesar_texto("The astronauts are traveling to mars in spaceships.", mostrar_traza=True)

Prueba de funcionamiento del pipeline:

[AUDITORÍA] Texto Original: The astronauts are traveling to mars in spaceships....
[AUDITORÍA] Pre-Stemming: ['astronauts', 'traveling', 'mars', 'spaceships']...
[AUDITORÍA] Post-Stemming: ['astronaut', 'travel', 'mar', 'spaceship']... (Total: 4)


In [109]:
class BuscadorManual:
    """
    Implementación artesanal de un Sistema de Recuperación de Información.
    Utiliza el Modelo de Espacio Vectorial (VSM) con ponderación TF-IDF.
    """
    def __init__(self):
        self.diccionario_idf = {}       # Almacena el peso IDF de cada término
        self.vectores_documentos = []   # Lista de vectores TF-IDF por documento
        self.normas_documentos = []     # Pre-cálculo de normas para optimizar el Coseno
        self.metadatos = []             # Referencia para mostrar título y sinopsis
        self.total_docs = 0             # Número total de documentos (N)
        
    def indexar(self, lista_textos, lista_titulos):
        """
        Genera el índice invertido y calcula los pesos vectoriales de la colección.
        """
        self.total_docs = len(lista_textos)
        self.metadatos = list(zip(lista_titulos, lista_textos))
        
        frecuencia_documental = Counter() # Conteo de en cuántos docs aparece cada término (DF)
        todos_los_tokens = []
        
        print(f"Iniciando indexación de {self.total_docs} documentos...")
        
        # FASE 1: Procesamiento y Cálculo de DF
        for i, texto in enumerate(lista_textos):
            # Se audita el primer documento para evidenciar el preprocesamiento en el examen
            es_primer_doc = (i == 0)
            if es_primer_doc:
                print("\n" + "="*50)
                print(" TRAZA DE INDEXACIÓN (DOCUMENTO #1)")
                print("="*50)
            
            tokens = procesar_texto(texto, mostrar_traza=es_primer_doc)
            todos_los_tokens.append(tokens)
            # Se usa set() para contar presencia binaria por documento
            frecuencia_documental.update(set(tokens))
            
        # FASE 2: Cálculo de IDF (Inverse Document Frequency)
        # Fórmula: log10( N / (df + 1) )
        print("\nCalculando pesos IDF para el vocabulario...")
        for termino, df in frecuencia_documental.items():
            self.diccionario_idf[termino] = math.log10(self.total_docs / (df + 1))
            
        # FASE 3: Generación de Vectores TF-IDF y Normas Euclidianas
        print("Construyendo espacio vectorial...")
        for tokens in todos_los_tokens:
            vector_actual = {}
            frecuencia_termino = Counter(tokens) # TF
            suma_cuadrados = 0
            
            for termino, tf in frecuencia_termino.items():
                if termino in self.diccionario_idf:
                    idf = self.diccionario_idf[termino]
                    peso = tf * idf
                    vector_actual[termino] = peso
                    suma_cuadrados += peso ** 2
            
            self.vectores_documentos.append(vector_actual)
            self.normas_documentos.append(math.sqrt(suma_cuadrados))
            
        print(f"¡Indexación completada! Vocabulario: {len(self.diccionario_idf)} términos.")

    def buscar(self, consulta, top_n=5):
        """
        Recupera documentos relevantes calculando la Similitud del Coseno
        entre el vector de la consulta y los vectores de los documentos.
        """
        print(f"\nProcesando consulta: '{consulta}'")
        
        # 1. Vectorización de la Consulta (Query)
        tokens_consulta = procesar_texto(consulta, mostrar_traza=True)
        conteo_consulta = Counter(tokens_consulta)
        vector_consulta = {}
        norma_consulta_cuadrada = 0
        
        for termino, tf in conteo_consulta.items():
            if termino in self.diccionario_idf:
                idf = self.diccionario_idf[termino]
                peso = tf * idf
                vector_consulta[termino] = peso
                norma_consulta_cuadrada += peso ** 2
        
        norma_consulta = math.sqrt(norma_consulta_cuadrada)
        if norma_consulta == 0: return []
            
        # 2. Cálculo de Similitud (Producto Punto / Normas)
        puntajes = []
        for i, vector_doc in enumerate(self.vectores_documentos):
            norma_doc = self.normas_documentos[i]
            if norma_doc == 0: continue
            
            producto_punto = 0
            for termino, peso_q in vector_consulta.items():
                if termino in vector_doc:
                    producto_punto += peso_q * vector_doc[termino]
            
            similitud = producto_punto / (norma_consulta * norma_doc)
            if similitud > 0:
                puntajes.append((i, similitud))
        
        # 3. Ordenamiento por relevancia descendente
        puntajes.sort(key=lambda x: x[1], reverse=True)
        
        resultados = []
        for idx, score in puntajes[:top_n]:
            titulo, info = self.metadatos[idx]
            resultados.append({"titulo": titulo, "score": score})
            
        return resultados

In [110]:
# --- Carga del Dataset ---
ruta_csv = '/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_movies.csv'
df_peliculas = pd.read_csv(ruta_csv)

# Se crea un campo 'contenido_texto' combinando título y descripción 
# para enriquecer el contexto semántico disponible para la búsqueda.
df_peliculas['contenido_texto'] = df_peliculas['movie_title'].fillna('') + " " + df_peliculas['movie_info'].fillna('')

# --- Instanciación e Indexación ---
buscador = BuscadorManual()

# Se utiliza el dataset completo para el entrenamiento
buscador.indexar(df_peliculas['contenido_texto'], df_peliculas['movie_title'])

Iniciando indexación de 17712 documentos...

 TRAZA DE INDEXACIÓN (DOCUMENTO #1)

[AUDITORÍA] Texto Original: Percy Jackson & the Olympians: The Lightning Thief Always tr...
[AUDITORÍA] Pre-Stemming: ['percy', 'jackson', 'olympians', 'lightning', 'thief', 'always', 'trouble', 'prone', 'life', 'teenager']...
[AUDITORÍA] Post-Stemming: ['perci', 'jackson', 'olympian', 'lightn', 'thief', 'alway', 'troubl', 'prone', 'life', 'teenag']... (Total: 49)

Calculando pesos IDF para el vocabulario...
Construyendo espacio vectorial...
¡Indexación completada! Vocabulario: 34903 términos.


In [111]:
consultas_examen = [
    "Películas sobre viajes espaciales",
    "Películas para ver en familia"
]

print("=== RESULTADOS DE LA SIMULACIÓN DE CONSULTAS ===")

for consulta in consultas_examen:
    print(f"\n{'='*30}\nCONSULTA: {consulta}\n{'='*30}")
    
    resultados = buscador.buscar(consulta, top_n=5)
    
    if not resultados:
        print(">> No se encontraron coincidencias relevantes.")
    else:
        for i, res in enumerate(resultados, 1):
            print(f"{i}. [Relevancia: {res['score']:.4f}] {res['titulo']}")

=== RESULTADOS DE LA SIMULACIÓN DE CONSULTAS ===

CONSULTA: Películas sobre viajes espaciales

Procesando consulta: 'Películas sobre viajes espaciales'

[AUDITORÍA] Texto Original: Películas sobre viajes espaciales...
[AUDITORÍA] Pre-Stemming: ['peliculas', 'sobre', 'viajes', 'espaciales']...
[AUDITORÍA] Post-Stemming: ['pelicula', 'sobr', 'viaj', 'espacial']... (Total: 4)
1. [Relevancia: 0.2133] All About My Mother (Todo sobre mi madre)

CONSULTA: Películas para ver en familia

Procesando consulta: 'Películas para ver en familia'

[AUDITORÍA] Texto Original: Películas para ver en familia...
[AUDITORÍA] Pre-Stemming: ['peliculas', 'para', 'ver', 'en', 'familia']...
[AUDITORÍA] Post-Stemming: ['pelicula', 'para', 'ver', 'en', 'familia']... (Total: 5)
1. [Relevancia: 0.2273] Sagrada: The Mystery of Creation
2. [Relevancia: 0.1628] Busco Novio Para Mi Mujer
3. [Relevancia: 0.1125] Cabin Boy
4. [Relevancia: 0.1082] Combat Shock (Fuerza en combate)
5. [Relevancia: 0.1073] What Have I Done t

In [113]:
# --- Función Auxiliar para Calcular Métricas ---
def calcular_precision_at_k(resultados_obtenidos, juicios_relevancia):
    """
    Calcula la Precisión@K comparando los resultados del sistema 
    con el juicio manual de relevancia.
    
    Args:
        resultados_obtenidos (list): Lista de diccionarios devuelta por el buscador.
        juicios_relevancia (list): Lista de 1s (relevante) y 0s (irrelevante) definida manualmente.
    
    Returns:
        float: Valor de precisión (0.0 a 1.0)
    """
    k = len(resultados_obtenidos)
    if k == 0: return 0.0
    
    # Solo tomamos los juicios hasta K (por si la lista es más larga)
    relevantes_recuperados = sum(juicios_relevancia[:k])
    
    return relevantes_recuperados / k

# --- SIMULACIÓN CON MÉTRICAS ---
consultas_examen = [
    {
        "texto": "Películas sobre viajes espaciales",
        "juicio_esperado": [0, 0, 0, 0, 0] 
    },
    {
        "texto": "Películas para ver en familia",
        "juicio_esperado": [1, 1, 1, 1, 1] 
    }
]

print("=== EVALUACIÓN CUANTITATIVA DEL SISTEMA (PRECISIÓN@5) ===")

promedio_precision = 0

for caso in consultas_examen:
    consulta = caso["texto"]
    juicios = caso["juicio_esperado"]
    
    print(f"\n{'='*40}\nCONSULTA: {consulta}\n{'='*40}")
    
    # Ejecutamos la búsqueda
    resultados = buscador.buscar(consulta, top_n=5)
    
    if not resultados:
        print(">> No se encontraron coincidencias.")
    else:
        # Mostramos resultados
        print(f"{'#':<3} {'Score':<10} {'Título'}")
        print("-" * 50)
        for i, res in enumerate(resultados):
            print(f"{i+1:<3} {res['score']:.4f}     {res['titulo']}")
            
        # Calculamos la métrica
        score_p5 = calcular_precision_at_k(resultados, juicios)
        promedio_precision += score_p5
        
        print("-" * 50)
        print(f">> Métrica Precision@5: {score_p5:.2f} ({score_p5*100}%)")
        print(f"   (Basado en juicio manual: {juicios})")

print("\n" + "="*40)
print(f"PRECISIÓN PROMEDIO DEL SISTEMA (MAP estimado): {(promedio_precision / len(consultas_examen)):.2f}")

=== EVALUACIÓN CUANTITATIVA DEL SISTEMA (PRECISIÓN@5) ===

CONSULTA: Películas sobre viajes espaciales

Procesando consulta: 'Películas sobre viajes espaciales'

[AUDITORÍA] Texto Original: Películas sobre viajes espaciales...
[AUDITORÍA] Pre-Stemming: ['peliculas', 'sobre', 'viajes', 'espaciales']...
[AUDITORÍA] Post-Stemming: ['pelicula', 'sobr', 'viaj', 'espacial']... (Total: 4)
#   Score      Título
--------------------------------------------------
1   0.2133     All About My Mother (Todo sobre mi madre)
--------------------------------------------------
>> Métrica Precision@5: 0.00 (0.0%)
   (Basado en juicio manual: [0, 0, 0, 0, 0])

CONSULTA: Películas para ver en familia

Procesando consulta: 'Películas para ver en familia'

[AUDITORÍA] Texto Original: Películas para ver en familia...
[AUDITORÍA] Pre-Stemming: ['peliculas', 'para', 'ver', 'en', 'familia']...
[AUDITORÍA] Post-Stemming: ['pelicula', 'para', 'ver', 'en', 'familia']... (Total: 5)
#   Score      Título
------------

**Análicis de resultados**
Para el caso de la primera consulta, tiene un 0% de precision ya que aparte de que solo muestra un resultado y muestra uno que a mi consideración no es relevante para la query. Por parte de la segunda en query en cambio los 5 resultados son correctos (relevantes para mi consideración). También hay que considerar que las consultas están en español y la mayoria de los documentos son en inglés. Como mejora se propone el mayor ciudado en el análisis del lenguaje tomando en cuenta los idiomas en los que se plantea trabajar y que el sistema no sea sensible ante eso.